<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/notebooks/41_ONNX_Advanced_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 41: Advanced ONNX Export (Dynamic Axes)

Standardowy `torch.onnx.export` zapamiętuje kształt danych przykładowych (`dummy_input`).
Jeśli w produkcji spróbujesz podać inne wymiary (np. większy batch, dłuższe zdanie, większy obrazek), ONNX rzuci błędem.

**Rozwiązanie: `dynamic_axes`**
To słownik, w którym mówimy eksporterowi:
*"Wymiar nr 0 to 'batch', on może się zmieniać. Wymiar nr 2 to 'height', też zmienny. Ale wymiar nr 1 (kanały) jest sztywny."*

Przetestujemy to na modelu RNN (gdzie zmienia się i Batch, i Długość Sekwencji).

In [1]:
import torch
import torch.nn as nn
import onnxruntime as ort
import numpy as np

# Konfiguracja
DEVICE = "cpu" # Do eksportu wystarczy CPU
FILE_NAME = "dynamic_rnn.onnx"

# Model: Prosty RNN
class DynamicRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x: [Batch, Seq_Len, Input_Size]
        out, _ = self.rnn(x)
        # Bierzemy ostatni krok czasu
        last_step = out[:, -1, :]
        return self.fc(last_step)

model = DynamicRNN(input_size=10, hidden_size=20, output_size=5)
model.eval()

print("Model RNN gotowy.")

Model RNN gotowy.


## Eksport Sztywny (Źle) vs Dynamiczny (Dobrze)

Najpierw przygotujmy `dummy_input`.
Niech to będzie `[Batch=1, Seq=5, Features=10]`.

Jeśli nie użyjemy `dynamic_axes`, ONNX "zabetonuje" `Seq=5`.
Jeśli użyjemy, będziemy mogli wrzucić sekwencję o długości 100.

In [10]:
import warnings
# Wyciszamy ostrzeżenia o przestarzałych funkcjach
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Dane przykładowe (do śledzenia)
dummy_input = torch.randn(1, 5, 10)

print(f"Eksportowanie z danymi: {dummy_input.shape}")

# EKSPORT Z DYNAMICZNYMI OSIAMI
torch.onnx.export(
    model,
    dummy_input,
    FILE_NAME,
    input_names=["input_seq"],
    output_names=["output_prob"],
    
    # --- TU DZIEJE SIĘ MAGIA ---
    dynamic_axes={
        "input_seq": {
            0: "batch_size",  # Oś 0 (Batch) jest zmienna
            1: "seq_len"      # Oś 1 (Długość czasu) jest zmienna
        },
        "output_prob": {
            0: "batch_size"   # Wyjście też musi mieć zmienny batch!
        }
    },
    # ---------------------------
    
    opset_version=17,
    
    # --- POPRAWKA ---
    # Wyłączamy silnik Dynamo, który próbuje "zabetonować" wymiary.
    # Wymuszamy użycie klasycznego TorchScript Tracing.
    dynamo=False  
)

print(f"✅ Wyeksportowano model do {FILE_NAME}")

Eksportowanie z danymi: torch.Size([1, 5, 10])
✅ Wyeksportowano model do dynamic_rnn.onnx


## Test Elastyczności (ONNX Runtime)

Teraz najważniejszy test.
Uruchomimy model w środowisku `onnxruntime` na danych o **zupełnie innych wymiarach** niż przy eksportcie.

1.  Zmienimy Batch z 1 na 3.
2.  Zmienimy Seq Len z 5 na 15.

Jeśli zadziała -> Sukces.
Jeśli wywali błąd -> Źle skonfigurowane osie.

In [8]:
# Tworzymy sesję
sess = ort.InferenceSession(FILE_NAME)

# 1. Test zgodny (1, 5, 10)
input_match = np.random.randn(1, 5, 10).astype(np.float32)
res_match = sess.run(None, {"input_seq": input_match})
print(f"Test zgodny (1, 5): ✅ OK. Wynik: {res_match[0].shape}")

# 2. Test dynamiczny (3, 15, 10) - INNY BATCH, INNA DŁUGOŚĆ
input_dynamic = np.random.randn(3, 15, 10).astype(np.float32)

try:
    res_dynamic = sess.run(None, {"input_seq": input_dynamic})
    print(f"Test dynamiczny (3, 15): ✅ OK. Wynik: {res_dynamic[0].shape}")
    print("Model obsłużył zmienne wymiary!")
except Exception as e:
    print(f"❌ BŁĄD: {e}")

Test zgodny (1, 5): ✅ OK. Wynik: (1, 5)
Test dynamiczny (3, 15): ✅ OK. Wynik: (3, 5)
Model obsłużył zmienne wymiary!


## Inspekcja Wejść

Możemy zapytać sesję ONNX, jakich kształtów się spodziewa.
Zamiast konkretnych liczb, powinniśmy zobaczyć symbole (np. `batch_size`, `seq_len` lub `unk__`).

In [6]:
inputs_info = sess.get_inputs()[0]

print("--- CO WIDZI ONNX? ---")
print(f"Nazwa wejścia: {inputs_info.name}")
print(f"Oczekiwany kształt: {inputs_info.shape}")

# Oczekujemy czegoś w stylu: ['batch_size', 'seq_len', 10]
# Jeśli widzisz [1, 5, 10], to znaczy, że dynamic_axes nie zadziałało.

--- CO WIDZI ONNX? ---
Nazwa wejścia: input_seq
Oczekiwany kształt: ['batch_size', 'seq_len', 10]


## 🥋 Black Belt Summary

1.  **Zawsze używaj `dynamic_axes`** dla Batch Size (oś 0). Nigdy nie wiesz, czy na produkcji będziesz przetwarzać 1 czy 64 zapytania naraz.
2.  **Dla NLP/Audio:** Zawsze dynamizuj oś czasu/sekwencji.
3.  **Dla Obrazów:** Tu ostrożnie. CNN często wymagają sztywnej wysokości/szerokości (chyba że używasz Global Pooling na końcu). Jeśli zmienisz rozmiar obrazka w locie, wagi mogą przestać pasować.